In [ ]:
#Idea of a data_collection dictionary to hold all the information. This is relevant if analysis is to be done in python instead of simply extracting a single branch using uproot.iterate

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import os
import scipy.optimize as sc
import matplotlib as mpl
import glob
from matplotlib.path import Path
from collections import Counter
from tqdm import tqdm

In [ ]:
datapath = "/home/haue/repositories/Masters/data/" 
Al_path = datapath + "bananas/Al"
P_path = datapath + "bananas/P"
Si_path = datapath + "bananas/Si"
Mg_path = datapath + "bananas/Mg"
#outputdir = "/figure_output/"
cuts0 = datapath + "cuts/idcuts.root" 
bancuts = datapath + "cuts/banana_cuts.root"
cuts1 = datapath + "cuts/totcuts.root"

Al_files = glob.glob(os.path.join(Al_path, "*mlio.root"))
P_files = glob.glob(os.path.join(P_path, "*mlio.root"))
Si_files = glob.glob(os.path.join(Si_path, "*mlio.root"))
Mg_files = glob.glob(os.path.join(Mg_path, "*mlio.root"))

In [3]:
# Storing the data in a dictionary for easy access

### Should be a more intuitive version than the ones used in Bananachecks...

data_files = {
    "Al": Al_files,
    "P": P_files,
    "Mg": Mg_files,
    "Si": Si_files
}
data_collection = {
      "Al": {},
    "P": {},
    "Mg": {},
    "Si": {},
    "All": {}
}

data_collection = {isotope: {f"id{i}": {key: [] for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]} for i in range(4)} for isotope in data_files}
data_collection["All"] = {f"id{i}": {key: [] for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]} for i in range(4)}


for isotope, files in data_files.items():
    #file_progress = tqdm(files, desc=f"Processing {isotope}", unit="file", total=len(files), dynamic_ncols=True)
    file_progress = tqdm(files, desc=f"Processing {isotope}", unit="file", total=len(files), bar_format="{l_bar}{bar} {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]")

    for file in file_progress:
        
        ## accesses the tree "a" in the root file
        r = uproot.open(file)["a"]
        ##Dummy array
        dum = r.arrays(["id", "E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"])
        
        ## converts it to an awkward array
        ids = ak.flatten(dum["id"])
        Es = ak.nan_to_num(ak.flatten(dum["E"]) / 1e3)
        fEdeps = ak.nan_to_num(ak.flatten(dum["fEdep"]) / 1e3)
        bEdeps = ak.nan_to_num(ak.flatten(dum["bEdep"]) / 1e3)
        angles = ak.nan_to_num(np.rad2deg(ak.flatten(dum["angle"])))
        Edeps = ak.nan_to_num(ak.flatten(dum["Edep"]) / 1e3)
        FIs = ak.flatten(dum["FI"])
        BIs = ak.flatten(dum["BI"])
        
        ##Creates entries in data_collection for each key and for each detector for each key
        for i in range(4):
            mask = (ids==i)
            detector_name = f"id{i}"
            data_collection[isotope][detector_name]["E"].extend(Es[mask])
            data_collection[isotope][detector_name]["fEdep"].extend(fEdeps[mask])
            data_collection[isotope][detector_name]["bEdep"].extend(bEdeps[mask])
            data_collection[isotope][detector_name]["angle"].extend(angles[mask])
            data_collection[isotope][detector_name]["Edep"].extend(Edeps[mask])
            data_collection[isotope][detector_name]["FI"].extend(FIs[mask])
            data_collection[isotope][detector_name]["BI"].extend(BIs[mask])
        file_progress.set_postfix({"Completed": f"{file_progress.n}/{file_progress.total}"})
    #print(f"Processing done for {isotope}!")
    

tqdm.write("Combining all isotopes into 'All'...")      
## Create the "All" branch in data_collection
for i in range(4):
    detector_key = f"id{i}"
    for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]:
        data_collection["All"][detector_key][key] = sum((data_collection[isotope][detector_key][key] for isotope in data_files), [])

tqdm.write("✅ All data combined successfully!")


Processing Al:   0%|           0/56 [00:00<?, ?file/s]

Processing Al: 100%|██████████ 56/56 [00:34<00:00,  1.64file/s]
Processing P: 100%|██████████ 63/63 [01:58<00:00,  1.89s/file]
Processing Mg: 100%|██████████ 12/12 [00:14<00:00,  1.18s/file]
Processing Si: 100%|██████████ 11/11 [00:28<00:00,  2.59s/file]


Combining all isotopes into 'All'...
✅ All data combined successfully!
